In [39]:
import classify
import numpy as np

from speech import *

# Supervised Logistic Regression + Bag of Words

In [46]:
print("Reading data")
tarfname = "data/speech.tar.gz"
speech = read_files(tarfname)
print("Training classifier")
import classify

Reading data
-- train data
train.tsv
4370
-- dev data
dev.tsv
414
-- transforming data and labels
Training classifier


In [73]:
# from sklearn.model_selection import GridSearchCV
# parameters = {
#         'vect__ngram_range': [(1, 1), (1, 2)],
#         'tfidf__use_idf': (True, False),
#         'clf__alpha': (1e-2, 1e-3),
#     }
# from sklearn.linear_model import LogisticRegression
# cls = LogisticRegression(random_state=0, penalty='l2', solver='lbfgs', tol=1e-6, max_iter=300, multi_class='multinomial')
# cls.fit(speech.trainX, speech.trainy)

def train_with_params(X, y, random_state=0, penalty='l2', solver='lbfgs', tol=1e-2, max_iter=1000, multi_class='auto', C=1.0): 
    from sklearn.linear_model import LogisticRegression
    cls = LogisticRegression(random_state=random_state, penalty=penalty, solver=solver, tol=tol, max_iter=max_iter, multi_class=multi_class)
    cls.fit(X, y)
    return cls

# list_C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
# for C in list_C:
#     cls = train_with_params(speech.trainX, speech.trainy, C=C)
#     print(C)
#     classify.evaluate(speech.trainX, speech.trainy, cls)
#     classify.evaluate(speech.devX, speech.devy, cls)

In [19]:
cls = classify.train_classifier(speech.trainX, speech.trainy)
print("Evaluating")
classify.evaluate(speech.trainX, speech.trainy, cls)
classify.evaluate(speech.devX, speech.devy, cls)

Reading data
-- train data
train.tsv
4370
-- dev data
dev.tsv
414
-- transforming data and labels
Training classifier


/home/junzel/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/junzel/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Evaluating
  Accuracy 0.9832951945080092
  Accuracy 0.4082125603864734


In [20]:
speech.trainX.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
speech.target_labels

array(['BACHMANN_PRIMARY2012', 'BIDEN_PRIMARY2008', 'CAIN_PRIMARY2012',
       'CLINTON_PRIMARY2008', 'EDWARDS_PRIMARY2008',
       'GINGRICH_PRIMARY2012', 'GIULIANI_PRIMARY2008',
       'HUCKABEE_PRIMARY2008', 'HUNTSMAN_PRIMARY2012',
       'MCCAIN_PRIMARY2008', 'OBAMA_PRIMARY2008', 'PAUL_PRIMARY2012',
       'PAWLENTY_PRIMARY2012', 'PERRY_PRIMARY2012',
       'RICHARDSON_PRIMARY2008', 'ROMNEY_PRIMARY2008',
       'ROMNEY_PRIMARY2012', 'SANTORUM_PRIMARY2012',
       'THOMPSON_PRIMARY2008'], dtype='<U22')

In [75]:
voc_dict = speech.count_vect.vocabulary_
for key in voc_dict.keys():
    if voc_dict[key] > 7000:
#         print(voc_dict[key])
#         print(key)

SyntaxError: unexpected EOF while parsing (<ipython-input-75-c478b255d383>, line 5)

In [84]:
from sklearn.decomposition import PCA
pca = PCA(n_components=414)
trainX_pca = pca.fit_transform(speech.trainX.toarray())

In [62]:
speech.trainX.shape

(4370, 7916)

In [82]:
speech.devX.shape

(414, 7916)

In [85]:
cls = train_with_params(pca.fit_transform(speech.trainX.toarray()), speech.trainy, max_iter=1000, tol=1e-9)
classify.evaluate(pca.fit_transform(speech.trainX.toarray()), speech.trainy, cls)
classify.evaluate(pca.fit_transform(speech.devX.toarray()), speech.devy, cls)

  Accuracy 0.5125858123569794
  Accuracy 0.16183574879227053


# Summarize so-far observations
## Tfidf
Using plain CountVectorizer, it is super easy to overfit to the training data. And development accuracy doesn't improve much with respect to different hyper-paramter settings.
Using TfidfVectorizer, the logistic regression model is hard to overfit on the training data (with penalty term). Give the best performance when setting penalty of logistic regression as 'none'.
## ngram
(1,2) gives the best development accuracy. (1,1) or (1,3) doesn't work as well as (1,2). 